In [1]:
# import packages #
import tkinter as tk
from tkinter import ttk
import requests
import pandas as pd
import sqlite3
import pymysql

#Define a function to transform json into dataframe
def sauto_json_to_df(sauto_data):
    return pd.json_normalize(sauto_data['results'])

# download data from Suato API# 
def request_sauto(page,limit=2000,manufacturer_model=0,condition=0,category_id=838,operating_lease='false'):
    time.sleep(0.5)
    manufacturer_models = {0:0,'audi':1,'bmw':2,'citroen':3,'fiat':4,'ford':5,'hyundai':6,'mazda':7,'mercedes-benz':8,'opel':9,'peugeot':10,'renault':11,'skoda':12,
                              'toyota':13,'volkswagen':14,'volvo':15} 
    conditions = {0:0,'nove':1,'ojete':2,'predvadeci':3,'havarovane':4,'veteran':5} 
    category_id=838
    url_template = 'https://www.sauto.cz/api/v1/items/search?limit={limit}&condition=condition&manufacturer_model_seo={manufacturer_model_seo}&category_id={category_id}&operating_lease=false'

    url = url_template.format(
        manufacturer_model_seo=manufacturer_models[manufacturer_model],
        category_id=category_id,
        limit = limit,
        page=page
    )
                           
    result = requests.get(url)

    return sauto_json_to_df(result.json())

def multiple_requests(beginning,ending,limit=2000,manufacturer_model=0,condition=0,category_id=838,operating_lease='false'):
    return pd.concat([
        request_sauto(i, limit = limit, manufacturer_model=manufacturer_model, condition=condition, 
                      category_id=category_id, operating_lease='false')
        for i in range(beginning,ending+1)
    ])

#Clean the data and write it to the database#
def writing_database():
    raw_data = multiple_requests(0,4)
    clean = raw_data[['manufacturer_cb.seo_name', 'model_cb.seo_name','fuel_cb.seo_name', 'price', 'tachometer','gearbox_cb.seo_name','manufacturing_date','locality.region','deal_type','create_date']]
    data_prague = clean[(clean['deal_type'] == 'sale')&(clean['locality.region']=='Hlavní město Praha')]
    data = data_prague[['manufacturer_cb.seo_name', 'model_cb.seo_name','fuel_cb.seo_name', 'price', 'tachometer','gearbox_cb.seo_name','manufacturing_date']]
   
    j_clean =data.to_json(orient = "records") 
    
    #Connect to our Alibaba Cloud server and write data to our database
    connection = pymysql.connect(host = '59.110.44.78',
                             user = 'root0',
                             password = 'root0',
                             db='SAUTO_DB')
    cur = connection.cursor()
    Querystatus='select status from SAUTO_DB.sauto_table where name = \'downloadFromAPI\''
    cur.execute(Querystatus)
    status = cur.fetchone()[0]
    
    #This is convenient for us to update the data. I have a 'status' in the database. When I switch to 1 and run this code, 
    #the database will be updated. The purpose of updating data ourselves is to save time for users, 
    #because it usually takes several minutes to wait, so we will update it regularly.
    if status == '1':
        
        updateStatus='update SAUTO_DB.sauto_table set status = \'1\' where name = \'downloadFromAPI\''
        cur.execute(updateStatus)
        sql='update SAUTO_DB.sauto_table set data = \''+j_clean+'\' where name = \'downloadFromAPI\''
        cur.execute(sql)
        connection.commit()
    
   


In [3]:
# design customer interface by tkinter# 
window = tk.Tk()
window.title('Welcome to choose your car in Prague')
window.geometry('1000x2000')
label1 = tk.Label(window,text='please enter your preference',bg='aliceblue',font = ('Arial',18),width = 30,height = 4)
label1.pack()


tk.Label(window, text='Fuel: ').place(x=50, y= 150)
tk.Label(window,text='Hint...choose Diesel by entering nafta : 1.nafta (Diesel), 2.benzin(petrol), 3.elektro(electro), 4.hybridni(hybrid), 5.lpg + benzin, 6.cng(nature gas + benzin)').place(x=50, y= 180)
tk.Label(window, text='Price(max): ').place(x=50, y= 210)
tk.Label(window, text='Tachometer(max）: ').place(x=50, y= 240)
tk.Label(window, text='Gearbox: ').place(x=50, y= 270)
tk.Label(window,text='Hint...choose automatic by entering automaticka : 1.automaticka(automatic), 2.manualni(manual)').place(x=50,y=300)
tk.Label(window, text='Manufacturing date: ').place(x=50, y= 330)
tk.Label(window,text='Hint...manufacturing date:2012-01-01').place(x=50,y=360)


entry1 = tk.StringVar()
entry_fuel = tk.Entry(window, textvariable=entry1)
entry_fuel.place(x=160, y=150)
entry2 = tk.IntVar()
entry_price = tk.Entry(window, textvariable=entry2)
entry_price.place(x=160, y=210)
entry3 = tk.IntVar()
entry_tachometer = tk.Entry(window, textvariable=entry3)
entry_tachometer.place(x=180, y=240)
entry4 = tk.StringVar()
entry_gearbox = tk.Entry(window, textvariable=entry4)
entry_gearbox.place(x=160, y=270)
entry5 = tk.StringVar()
entry_manufacturing = tk.Entry(window, textvariable=entry5)
entry_manufacturing.place(x=200, y=330)

# fetch data from database# 
def reading_database():
    connection = pymysql.connect(host = '59.110.44.78',
                             user = 'root0',
                             password = 'root0',
                             db='SAUTO_DB')
    

    read = 'select data from SAUTO_DB.sauto_table where name = \'downloadFromAPI\''
    cur = connection.cursor()
    cur.execute(read)
    data_json = cur.fetchone()[0]
    return pd.read_json(data_json)

# define customer's buying parameters # 
def selection(fuel='Nafta',price=1000000,tachometer=150000,gearbox='Manuální',date='2012-01-01'):
    
    df_database = reading_database()
    df= df_database[(df_database['fuel_cb.seo_name'] == fuel) & (df_database['gearbox_cb.seo_name'] == gearbox)&(df_database['price']<price)&(df_database['tachometer']<tachometer)&(df_database['manufacturing_date']>date)]
    new_df = df.drop_duplicates()
    return new_df.sort_values(['price','tachometer']).head()

#Filter and present visual results by obtaining user parameters
def get_choice():
    fuel = entry1.get()
    price = entry2.get()
    tachometer = entry3.get()
    gearbox = entry4.get()
    manufacturing = entry5.get()
    a = selection(fuel,price,tachometer,gearbox,manufacturing)
    print(a)
    cols = list(a.columns)
    tree = ttk.Treeview(window)
    tree.pack()
    tree["columns"] = cols
    for i in cols:
        tree.column(i, anchor="w")
        tree.heading(i, text=i, anchor='w')

    for index, row in a.iterrows():
        tree.insert("",0,text=index,values=list(row))

    
    
btn_get_choice = tk.Button(window, text='Get your choice!', command=get_choice)
btn_get_choice.place(x=300, y=430)


window.mainloop()    
    
    

    manufacturer_cb.seo_name model_cb.seo_name fuel_cb.seo_name   price  \
155                    skoda             rapid            nafta  209000   
400                  hyundai               i30            nafta  245000   
182                   nissan           qashqai            nafta  250000   
386                    skoda             rapid            nafta  259000   
644                    skoda             fabia            nafta  282000   

     tachometer gearbox_cb.seo_name manufacturing_date  
155       96261            manualni         2015-01-01  
400       95031            manualni         2016-01-01  
182       66993            manualni         2013-01-01  
386       96717            manualni         2015-01-01  
644       38015            manualni         2017-01-01  
